## ContentHandler

In [ ]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
from typing import Dict


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

## IndicatorsQuestionsChain

In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts import PromptTemplate


class IndicatorsQuestionsChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "indicators_questions_chain"

    @classmethod
    def from_indicators(
        cls,
        indicators: str,
        **kwargs: Any,
    ) -> IndicatorsQuestionsChain:
        PROMPT_TEMPLATE = (
            """The user's question may be to ask the latest market trend of a certain cryptocurrency. 
The following index tools can help users analyze market trend.
Index tool's names are """
            + indicators
        )
        PROMPT_TEMPLATE = (
            PROMPT_TEMPLATE
            + """\nPlease generats questions, to ask the latest index above of the cryptocurrency with its symbol in user's question.
Note that some index are composed of a set of numerical values. So you should generate the questions ask to get all values of the index.
You should generate the questions in JSON Object format, and use index tool's name as JSON object's key, use the question which you generated as the value of JSON object.
Do not need any hint, just a JSON object.
User's Question: {question}
You generations:"""
        )
        # Finally, add a item into the JSON object, key is "question", use the following user's question as its value.

        prompt = PromptTemplate(input_variables=["question"], template=PROMPT_TEMPLATE)
        content_handler = ContentHandler()
        llm = SagemakerEndpoint(
            endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
            # credentials_profile_name="default",
            region_name="us-east-1",
            model_kwargs={
                "parameters": {
                    "do_sample": True,
                    # "top_p": 0.9,
                    # "top_k": 10,
                    "repetition_penalty": 1.03,
                    "max_new_tokens": 1024,
                    "temperature": 0.1,
                    "return_full_text": False,
                    # "max_length":2048,
                    "truncate": 2048,
                    # "num_return_sequences":2000,
                    # "stop": ["\n"],
                },
            },
            content_handler=content_handler,
            verbose=kwargs["verbose"],
        )
        return cls(llm=llm, prompt=prompt, **kwargs)

## Llama2APIChain

In [ ]:
from langchain.chains import APIChain
from langchain import LLMChain
from langchain.requests import TextRequestsWrapper
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint


class Llama2APIChain(APIChain):
    @property
    def _chain_type(self) -> str:
        return "llama2_api_chain"

    @classmethod
    def from_docs(
        cls,
        api_docs: str,
        headers: any,
        **kwargs: any,
    ) -> Llama2APIChain:
        content_handler = ContentHandler()

        llm_gen_url = SagemakerEndpoint(
            endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
            # credentials_profile_name="default",
            region_name="us-east-1",
            model_kwargs={
                "parameters": {
                    "do_sample": True,
                    # "top_p": 0.9,
                    # "top_k": 10,
                    "repetition_penalty": 1.03,
                    "max_new_tokens": 1024,
                    "temperature": 0.1,
                    "return_full_text": False,
                    # "max_length":2048,
                    "truncate": 2048,
                    # "num_return_sequences":2000,
                    # "stop": ["\n"],
                },
            },
            content_handler=content_handler,
            verbose=kwargs["verbose"],
        )

        llm_gen_resp = SagemakerEndpoint(
            endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
            # credentials_profile_name="default",
            region_name="us-east-1",
            model_kwargs={
                "parameters": {
                    "do_sample": True,
                    "top_p": 0.9,
                    "top_k": 10,
                    "repetition_penalty": 1.03,
                    "max_new_tokens": 1024,
                    "temperature": 0.9,
                    "return_full_text": False,
                    # "max_length":2048,
                    "truncate": 2048,
                    # "num_return_sequences":2000,
                    # "stop": ["\nHuman:"],
                },
            },
            content_handler=content_handler,
            verbose=kwargs["verbose"],
        )

        API_URL_PROMPT_TEMPLATE = """API Documentation:
{api_docs}

According above documentation, the full API url was generated to call for answering the question bellow.
The full API url in order to get a response that is as short as possible. And the full API url to deliberately exclude any unnecessary pieces of data in the API call.
You should not generate any hint. Just generate the full API url to answer the question as far as possible.

Question:{question}

API url:"""
        API_URL_PROMPT = PromptTemplate(
            input_variables=[
                "api_docs",
                "question",
            ],
            template=API_URL_PROMPT_TEMPLATE,
        )
        API_RESPONSE_PROMPT_TEMPLATE = (
            API_URL_PROMPT_TEMPLATE
            + """ {api_url}
    
Here is the response from the API:

{api_response}

Summarize this response to answer the original question.
Please include all data in the response.

Summary:"""
        )
        API_RESPONSE_PROMPT = PromptTemplate.from_template(API_RESPONSE_PROMPT_TEMPLATE)
        return cls(
            api_request_chain=LLMChain(
                llm=llm_gen_url, prompt=API_URL_PROMPT, **kwargs
            ),
            api_answer_chain=LLMChain(
                llm=llm_gen_resp, prompt=API_RESPONSE_PROMPT, **kwargs
            ),
            api_docs=api_docs,
            requests_wrapper=TextRequestsWrapper(headers=headers),
            **kwargs,
        )

## MarketTrendChain

In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts import PromptTemplate


class MarketTrendChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "indicators_questions_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> MarketTrendChain:
        PROMPT_TEMPLATE = """Content
```
{data}
```

Question
```
{question}
```


Please organize the above content into a complete paragraph first. Then use the content to answer the question. Finally, give your anlysis from the content, and give some investment advise as far as possible. 

"""
        prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
        return cls(llm=llm, prompt=prompt, **kwargs)

In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional
import datetime
from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


class TradingViewReasearchReportChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        # answer=self.answer_chain.run(question=inputs['user_input'],context=res)
        if run_manager:
            run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "indicators_questions_chain"

    @classmethod
    def from_llm(
        cls,
        llm,
        **kwargs: Any,
    ) -> TradingViewReasearchReportChain:
        time = f"It is {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} now.\n"
        PROMPT_TEMPLATE = (
            time
            + """We have an unofficial python API wrapper to retrieve technical analysis from TradingView.
Retrieving the analysis:
- summary: Technical analysis (based on both oscillators and moving averages).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 12, 'SELL': 7, 'NEUTRAL': 9}}
```
- oscillators: Technical analysis (based on oscillators).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 2, 'SELL': 1, 'NEUTRAL': 8, 'COMPUTE': {{'RSI': 'NEUTRAL', 'STOCH.K': 'NEUTRAL', 'CCI': 'NEUTRAL', 'ADX': 'NEUTRAL', 'AO': 'NEUTRAL', 'Mom': 'BUY', 'MACD': 'SELL', 'Stoch.RSI': 'NEUTRAL', 'W%R': 'NEUTRAL', 'BBP': 'BUY', 'UO': 'NEUTRAL'}}}}
```
- moving_averages: Technical analysis (based on moving averages).
```
# Example
{{'RECOMMENDATION': 'BUY', 'BUY': 9, 'SELL': 5, 'NEUTRAL': 1, 'COMPUTE': {{'EMA10': 'SELL', 'SMA10': 'SELL', 'EMA20': 'SELL', 'SMA20': 'SELL', 'EMA30': 'BUY', 'SMA30': 'BUY', 'EMA50': 'BUY', 'SMA50': 'BUY', 'EMA100': 'BUY', 'SMA100': 'BUY', 'EMA200': 'BUY', 'SMA200': 'BUY', 'Ichimoku': 'NEUTRAL', 'VWMA': 'SELL', 'HullMA': 'BUY'}}}}
```
- indicators: Technical indicators.
```
# Example
{{'Recommend.Other': 0, 'Recommend.All': 0.26666667, 'Recommend.MA': 0.53333333, 'RSI': 60.28037412, 'RSI[1]': 58.58364778, 'Stoch.K': 73.80404453, 'Stoch.D': 79.64297643, 'Stoch.K[1]': 78.88160227, 'Stoch.D[1]': 85.97647064, 'CCI20': 46.58442886, 'CCI20[1]': 34.57058796, 'ADX': 35.78754863, 'ADX+DI': 23.16948389, 'ADX-DI': 13.82449817, 'ADX+DI[1]': 24.15991909, 'ADX-DI[1]': 13.87125505, 'AO': 6675.72158824, 'AO[1]': 7283.92420588, 'Mom': 1532.6, 'Mom[1]': 108.29, 'MACD.macd': 2444.73734978, 'MACD.signal': 2606.00138275, 'Rec.Stoch.RSI': 0, 'Stoch.RSI.K': 18.53740187, 'Rec.WR': 0, 'W.R': -26.05634845, 'Rec.BBPower': 0, 'BBPower': 295.52055898, 'Rec.UO': 0, 'UO': 55.68311917, 'close': 45326.97, 'EMA5': 45600.06414333, 'SMA5': 45995.592, 'EMA10': 45223.22433151, 'SMA10': 45952.635, 'EMA20': 43451.52018338, 'SMA20': 43609.214, 'EMA30': 41908.5944052, 'SMA30': 40880.391, 'EMA50': 40352.10222373, 'SMA50': 37819.3566, 'EMA100': 40356.09177879, 'SMA100': 38009.7808, 'EMA200': 39466.50411569, 'SMA200': 45551.36135, 'Rec.Ichimoku': 0, 'Ichimoku.BLine': 40772.57, 'Rec.VWMA': 1, 'VWMA': 43471.81729377, 'Rec.HullMA9': -1, 'HullMA9': 45470.37107407, 'Pivot.M.Classic.S3': 11389.27666667, 'Pivot.M.Classic.S2': 24559.27666667, 'Pivot.M.Classic.S1': 33010.55333333, 'Pivot.M.Classic.Middle': 37729.27666667, 'Pivot.M.Classic.R1': 46180.55333333, 'Pivot.M.Classic.R2': 50899.27666667, 'Pivot.M.Classic.R3': 64069.27666667, 'Pivot.M.Fibonacci.S3': 24559.27666667, 'Pivot.M.Fibonacci.S2': 29590.21666667, 'Pivot.M.Fibonacci.S1': 32698.33666667, 'Pivot.M.Fibonacci.Middle': 37729.27666667, 'Pivot.M.Fibonacci.R1': 42760.21666667, 'Pivot.M.Fibonacci.R2': 45868.33666667, 'Pivot.M.Fibonacci.R3': 50899.27666667, 'Pivot.M.Camarilla.S3': 37840.08, 'Pivot.M.Camarilla.S2': 39047.33, 'Pivot.M.Camarilla.S1': 40254.58, 'Pivot.M.Camarilla.Middle': 37729.27666667, 'Pivot.M.Camarilla.R1': 42669.08, 'Pivot.M.Camarilla.R2': 43876.33, 'Pivot.M.Camarilla.R3': 45083.58, 'Pivot.M.Woodie.S3': 21706.84, 'Pivot.M.Woodie.S2': 25492.42, 'Pivot.M.Woodie.S1': 34876.84, 'Pivot.M.Woodie.Middle': 38662.42, 'Pivot.M.Woodie.R1': 48046.84, 'Pivot.M.Woodie.R2': 51832.42, 'Pivot.M.Woodie.R3': 61216.84, 'Pivot.M.Demark.S1': 35369.915, 'Pivot.M.Demark.Middle': 38908.9575, 'Pivot.M.Demark.R1': 48539.915, 'open': 44695.95, 'P.SAR': 48068.64, 'BB.lower': 37961.23510877, 'BB.upper': 49257.19289123, 'AO[2]': 7524.31223529, 'volume': 32744.424503, 'change': 1.44612354, 'low': 44203.28, 'high': 45560}}
```

We got the analysis data of {symbol} from python-tradingview-ta as following:
summary:{summary}
oscillators:{oscillators}
moving_averages:{moving_averages}
indicators:{indicators}
"""
        )
        PROMPT_TEMPLATE = (
            PROMPT_TEMPLATE
            + """\nPlease generate the analysis results by analyzing data the above, and provide the market trend.
Your generation:
"""
        )

        prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
        return cls(llm=llm, prompt=prompt, **kwargs)

In [ ]:
from __future__ import annotations
from typing import Any, Dict, List, Optional
from pydantic import Extra
from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import AsyncCallbackManagerForChainRun,CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

import taapi_docs
import cmc_api_docs

import os

class MarketTrendAndInvestmentAdviseToolChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    iq_chain:IndicatorsQuestionsChain

    rsi_chain : Llama2APIChain
    cci_chain : Llama2APIChain
    dmi_chain : Llama2APIChain
    psar_chain : Llama2APIChain
    stochrsi_chain : Llama2APIChain
    cmf_chain : Llama2APIChain

    latest_quote_chain:Llama2APIChain

    mt_chain:MarketTrendChain

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
             run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response =  self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
             run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        question=response.generations[0][0].text
        indicator_questions_json =  self.iq_chain.run(question)
        index_questions = json.loads(indicator_questions_json)
        
        rsi_res =  self.rsi_chain.run(index_questions["RSI"])
        cci_res =  self.cci_chain.run(index_questions["CCI"])
        dmi_res =  self.dmi_chain.run(index_questions["DMI"])
        psar_res =  self.psar_chain.run(index_questions["PSAR"])
        stochrsi_res =  self.stochrsi_chain.run(index_questions["STOCHRSI"])
        cmf_res =  self.cmf_chain.run(index_questions["CMF"])
        latest_quote_res= self.latest_quote_chain.run(question)
        
        data=rsi_res+"\n"+cci_res+"\n"+dmi_res+"\n"+psar_res+"\n"+stochrsi_res+"\n"+cmf_res+"\n"+latest_quote_res
        market_trend_res= self.mt_chain.run(question=question,data=data)

        return {self.output_key: market_trend_res}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text(
                response.generations[0][0].text,
                color="yellow",
                end="\n",
                verbose=self.verbose,
            )
        question=response.generations[0][0].text
        indicator_questions_json = await self.iq_chain.arun(question)
        index_questions = json.loads(indicator_questions_json)
        
        rsi_res = await self.rsi_chain.arun(index_questions["RSI"])
        cci_res = await self.cci_chain.arun(index_questions["CCI"])
        dmi_res = await self.dmi_chain.arun(index_questions["DMI"])
        psar_res = await self.psar_chain.arun(index_questions["PSAR"])
        stochrsi_res = await self.stochrsi_chain.arun(index_questions["STOCHRSI"])
        cmf_res = await self.cmf_chain.arun(index_questions["CMF"])
        latest_quote_res=await self.latest_quote_chain.arun(question)

        data=rsi_res+"\n"+cci_res+"\n"+dmi_res+"\n"+psar_res+"\n"+stochrsi_res+"\n"+cmf_res+"\n"+latest_quote_res
        market_trend_res=await self.mt_chain.arun(question=question,data=data)

        return {self.output_key: market_trend_res}

    @property
    def _chain_type(self) -> str:
        return "market_trend_investment_advise_chain"

    @classmethod
    def from_create(
        cls,
        **kwargs: Any,
    ) -> MarketTrendAndInvestmentAdviseToolChain:
        prompt_template = (
"""User may ask some cryptocurrency's market trend.
Please generate a complete question in English, using user's input below.
User's input:{user_input}
Complete question:"""
)
        content_handler = ContentHandler()
        llama2_01 = SagemakerEndpoint(
            endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
            # credentials_profile_name="default",
            region_name="us-east-1",
            model_kwargs={
                "parameters": {
                    "do_sample": True,
                    # "top_p": 0.9,
                    # "top_k": 10,
                    "repetition_penalty": 1.03,
                    "max_new_tokens": 1024,
                    "temperature": 0.1,
                    "return_full_text": False,
                    # "max_length":2048,
                    "truncate": 2048,
                    # "num_return_sequences":2000,
                    # "stop": ["\n"],
                },
            },
            content_handler=content_handler,
            verbose=kwargs["verbose"],
        )
        llama2_09 = SagemakerEndpoint(
            endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
            # credentials_profile_name="default",
            region_name="us-east-1",
            model_kwargs={
                "parameters": {
                    "do_sample": True,
                    # "top_p": 0.9,
                    # "top_k": 10,
                    "repetition_penalty": 1.03,
                    "max_new_tokens": 1024,
                    "temperature": 0.9,
                    "return_full_text": False,
                    # "max_length":2048,
                    "truncate": 2048,
                    # "num_return_sequences":2000,
                    # "stop": ["\n"],
                },
            },
            content_handler=content_handler,
            verbose=kwargs["verbose"],
        )
        prompt = PromptTemplate.from_template(prompt_template)

        iq_chain = IndicatorsQuestionsChain.from_indicators(
            indicators="RSI,CCI,DMI,PSAR,STOCHRSI,CMF", **kwargs, 
        )

        taapi_key = os.getenv("TAAPI_KEY")
        headers = {
            "Accepts": "application/json",
        }

        rsi_api_docs = PromptTemplate.from_template(taapi_docs.RSI_API_DOCS).format(
            taapi_key=taapi_key
        )
        cci_api_docs = PromptTemplate.from_template(taapi_docs.CCI_API_DOCS).format(
            taapi_key=taapi_key
        )
        dmi_api_docs = PromptTemplate.from_template(taapi_docs.DMI_API_DOCS).format(
            taapi_key=taapi_key
        )
        psar_api_docs = PromptTemplate.from_template(taapi_docs.PSAR_API_DOCS).format(
            taapi_key=taapi_key
        )
        stochrsi_api_docs = PromptTemplate.from_template(taapi_docs.STOCHRSI_API_DOCS).format(
            taapi_key=taapi_key
        )
        cmf_api_docs = PromptTemplate.from_template(taapi_docs.CMF_API_DOCS).format(
            taapi_key=taapi_key
        )
        rsi_chain = Llama2APIChain.from_docs(
            api_docs=rsi_api_docs, headers=headers,**kwargs, 
        )
        cci_chain = Llama2APIChain.from_docs(
            api_docs=cci_api_docs, headers=headers, **kwargs,
        )
        dmi_chain = Llama2APIChain.from_docs(
            api_docs=dmi_api_docs, headers=headers, **kwargs,
        )
        psar_chain = Llama2APIChain.from_docs(
            api_docs=psar_api_docs, headers=headers, **kwargs,
        )
        stochrsi_chain = Llama2APIChain.from_docs(
            api_docs=stochrsi_api_docs, headers=headers, **kwargs,
        )
        cmf_chain = Llama2APIChain.from_docs(
            api_docs=cmf_api_docs, headers=headers, **kwargs,
        )

        headers = {
          'Accepts': 'application/json',
          'X-CMC_PRO_API_KEY': os.getenv("CMC_API_KEY"),
        }
        latest_quote_chain = Llama2APIChain.from_docs(
            api_docs=cmc_api_docs.CMC_QUOTE_LASTEST_API_DOC, headers=headers, **kwargs,
        )

        gpt4 = ChatOpenAI(
            model="gpt-4",
            temperature=0.9,
            **kwargs,
        )
        mt_chain=MarketTrendChain.from_llm(llm=gpt4,**kwargs)

        return cls(
            llm=llama2_01,
            iq_chain=iq_chain, 
            rsi_chain=rsi_chain,
            cci_chain=cci_chain,
            dmi_chain=dmi_chain,
            psar_chain=psar_chain,
            stochrsi_chain=stochrsi_chain,
            cmf_chain=cmf_chain,
            latest_quote_chain=latest_quote_chain,
            mt_chain=mt_chain,
            prompt=prompt, 
            **kwargs,)

swftc(swftcoin) latest quote

In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

mtia_chain = MarketTrendAndInvestmentAdviseToolChain.from_create(verbose=True)
user_input = input("Your prompt: ")
res = mtia_chain.run(user_input)